# Set Covering - 2023-10-10
Copyright(c) 2023 Alex Buffa


In [184]:
import numpy as np
from random import random
from typing import Tuple, Set
from functools import reduce
from operator import or_
from queue import PriorityQueue, LifoQueue, SimpleQueue, Queue
from collections import namedtuple
from typing import Callable
from math import ceil
from tqdm.notebook import tqdm
Result = namedtuple("Result", ["name", "iters", "taken", "coverage", "prio"])
State = Tuple[Set[int], Set[int]]

Define our problem data

In [185]:
PROBLEM_SIZE = 10
NUM_SETS = 30
THRESHOLD = 0.3
SETS = tuple(np.array([random() < THRESHOLD for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
# Redefine SETS until the problem is solvable
while not all(reduce(or_, [SETS[i] for i in range(NUM_SETS)])):
    SETS = tuple(np.array([random() < THRESHOLD for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
results: dict[str, Result] = dict()


In [186]:
# Utility function just to see our current taken array
def visualize_state(state: State) -> list[int]:
    return sum([SETS[i] for i in state[0]])

In [187]:
def goal_check(state: State):
    return all(reduce(or_, [SETS[i] for i in state[0]], np.array([False for _ in range(PROBLEM_SIZE)])))

In [188]:
def search(name: str, initial_state: State = None,*, frontier: "Queue" = None, priority: Callable[[State],int] = None) -> Result:
    """Generic Search Function.
    Through the parameters 
    """
    if initial_state is None:
        initial_state = (set(), set(range(NUM_SETS)))
    assert len(initial_state) == 2, "Invalid State"
    if frontier is None:
        frontier = PriorityQueue()
    if priority is None:
        priority = lambda _: None
    WrappedState = namedtuple("WrappedState", ["priority", "state"])
    frontier.put(WrappedState(priority(initial_state), initial_state))
    _, state = frontier.get()
    counter = 0
    with tqdm(total=None) as pbar:
        while not goal_check(state):
            counter += 1
            for a in state[1]:
                new_state = (state[0] ^ {a}, state[1] ^ {a})
                frontier.put(WrappedState(priority(new_state), new_state))
            _, state = frontier.get()
            pbar.update()
    res = Result(name, counter, state[0], visualize_state(state), priority(state))
    results[name] = res
    return res


## Depth First Search

In [189]:
search(name="Depth-First", frontier=LifoQueue()).taken

0it [00:00, ?it/s]

{20, 21, 22, 23, 24, 25, 26, 27, 28, 29}

## Breadth First Search

In [190]:
# Using SimpleQueue, which does it internally
search(name="Breadth-First", frontier=SimpleQueue()).taken

0it [00:00, ?it/s]

{0, 6, 14}

## Djikstra Search

In [191]:
def cost(state: State) -> int:
    """Number of sets"""
    return len(state[0]) # + sum(abs(np.ones(PROBLEM_SIZE) - sum([SETS[i] for i in state[0]])))

In [192]:
search(name="Djikstra", priority=cost).taken

0it [00:00, ?it/s]

{9, 20, 26}

## A* Search

A* requires a heuristic function that is admissible, i.e. it never overestimates the cost to reach the goal.
For example, we define the distance function as the optimal number of sets that are needed to cover the missing tiles.    
With the above distance function we have an admissible heuristic function.  
The priority for A* is given by the sum of the cost function and the heuristic function.

In [193]:
def distance(state: State) -> int:
    max_size = max(sum(s) for i, s in enumerate(SETS) if i in state[1])
    if(len(state[0]) == 0 ):
        return ceil(PROBLEM_SIZE/max_size)
    return ceil((sum([SETS[i] for i in state[0]]) == 0).sum() / max_size)

In [194]:
search(name="A*", priority=lambda x: cost(x) + distance(x)).taken

0it [00:00, ?it/s]

{1, 20, 29}

In [195]:
print("All the results obtained above, sorted by number of iterations")
for result in reversed(results.values()):
    print(result)

All the results obtained above, sorted by number of iterations
Result(name='A*', iters=42, taken={1, 20, 29}, coverage=array([1, 1, 1, 1, 1, 1, 2, 2, 1, 1]), prio=3)
Result(name='Djikstra', iters=925, taken={9, 26, 20}, coverage=array([1, 2, 1, 1, 2, 1, 1, 2, 1, 1]), prio=3)
Result(name='Breadth-First', iters=1053, taken={0, 14, 6}, coverage=array([1, 2, 2, 2, 1, 1, 1, 1, 1, 1]), prio=None)
Result(name='Depth-First', iters=10, taken={20, 21, 22, 23, 24, 25, 26, 27, 28, 29}, coverage=array([3, 4, 1, 3, 3, 2, 2, 3, 3, 3]), prio=None)
